# Equal-Weight S&P 500 Index Fund

## Introduction 


The goal of this project is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund. Since IEX requires a paid key for real stock data, this is not exact.


In [23]:
import numpy as np 
import pandas as pd
import requests # Basically, requests data from the internet 
import xlsxwriter
import math

In [24]:
# Acquiring an API token 
stocks = pd.read_csv('sp_500_stocks.csv') # Stores CSV in a df

In [25]:
# Acquiring API token
from secrets import IEX_CLOUD_API_TOKEN

## API Call

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [26]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'N)G SAASLOAEEALR(LCN KGSD/ BMTQTE', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'ocilaiff', 'close': None, 'closeTime': None, 'closeSource': 'aicolfif', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 130.109, 'latestSource': 'IEX Last Trade', 'latestTime': 'December 16, 2020', 'latestUpdate': 1613998899849, 'latestVolume': None, 'iexRealtimePrice': 133.913, 'iexRealtimeSize': 3, 'iexLastUpdated': 1614442181612, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 133.88, 'previousVolume': 159258140, 'change': -0.168, 'changePercent': -0.00131, 'volume': None, 'iexMarketPercent': 0.0165733783818506, 'iexVolume': 1654914, 'avgTotalVolume': 976

## Parsing Our API Call

In [27]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/1000000000000)

2.183937247761


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [76]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

In [77]:
final_dataframe.append(
    pd.Series(
    [
            symbol, 
            price, 
            market_cap, 
            'N/A'
    ],
    index = my_columns # Tells append method which columsn to add the data to
    ), 
    ignore_index = True # This statement is common in Pandas
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,ZTS,130.109,2183937247761,N/A


## Using Batch API Calls to Improve Performance

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [79]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [80]:
# Remember that stocks would give the whole data frame, w/ ticker gives us the series
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,120.360,37699018154,N/A
1,AAL,17.524,10664499611,N/A
2,AAP,166.360,11111092086,N/A
3,AAPL,128.056,2223440393724,N/A
4,ABBV,107.960,186135073391,N/A
...,...,...,...,...
500,YUM,109.830,33061606525,N/A
501,ZBH,152.250,31194909826,N/A
502,ZBRA,393.880,20761631878,N/A
503,ZION,42.410,7104823251,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [81]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:100000000


In [82]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,120.360,37699018154,1645
1,AAL,17.524,10664499611,11299
2,AAP,166.360,11111092086,1190
3,AAPL,128.056,2223440393724,1546
4,ABBV,107.960,186135073391,1834
...,...,...,...,...
500,YUM,109.830,33061606525,1802
501,ZBH,152.250,31194909826,1300
502,ZBRA,393.880,20761631878,502
503,ZION,42.410,7104823251,4669


## Formatting Our Excel Output


### Initializing our XlsxWriter Object

In [134]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating Formats

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [135]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color':background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color':background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color':background_color,
        'border': 1
    }
)

In [136]:
column_formats = {
    'A': ['Ticker', string_format], 
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}




for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [139]:
# Saving Excel Output
writer.save()